In [ ]:
import os
import numpy as np
from matplotlib import pyplot as plt
import rasterio
import rasterio.plot
from tqdm.notebook import tqdm

# 1. Data Loading and Visualization of Fires

In [ ]:
# TODO: choose different tif - one with a FIRE
image_path = "/kaggle/input/wildfirespreadts/2018/fire_21890058/2018-07-18.tif"
tiff = rasterio.open(image_path)
rasterio.plot.show(tiff, title = image_path)
plt.show()

In [ ]:
labels = list(tiff.descriptions)
print(labels)

In [ ]:
fig, axs = plt.subplots(4,6, figsize=(16,16))
for idx, (ax, label) in enumerate(zip(axs.flatten(), tiff.descriptions)):
  im = np.expand_dims(img[idx],-1).astype(int)
  ax.imshow(im)
  ax.set_title(label)
axs.flatten()[-1].remove()
plt.tight_layout()
plt.show()

In [ ]:
import imageio
import rasterio
import os
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import Image, display

def create_band_gif(folder_path, band_idx, label, output_gif='output.gif', duration=0.5, background_value = 15):
    """
    Creates and displays a GIF from all TIFF files in a folder for a specific band.
    
    Args:
        folder_path (str): Path to the folder containing TIFF files.
        band_idx (int): Index of the band to use.
        label (str): Label for the GIF frames.
        output_gif (str): Output path for the GIF.
        duration (float): Time between frames in seconds.
    """
    images = []
    tiff_files = [f for f in os.listdir(folder_path) if f.endswith('.tif') or f.endswith('.tiff')]

    for file in sorted(tiff_files):
        file_path = os.path.join(folder_path, file)
        tiff, img, labels = read_tiff_file(file_path)
        
        if band_idx >= img.shape[0]:
            print(f"Band index {band_idx} out of range for file {file}")
            continue
        
        # Extract the specific band
        band_img = img[band_idx]
        band_img[np.isnan(band_img)] = background_value
        # print(band_img.min(), band_img.max())
        # Plot the band
        fig, ax = plt.subplots(figsize=(6,6))
        ax.imshow(band_img, cmap='inferno')
        ax.set_title(f"{label} - {file}")
        ax.axis('off')
        
        # Save plot to an image buffer
        fig.canvas.draw()
        image = np.frombuffer(fig.canvas.tostring_rgb(), dtype='uint8')
        image = image.reshape(fig.canvas.get_width_height()[::-1] + (3,))
        images.append(image)
        plt.close(fig)
    
    # Create and save the GIF
    if images:
        imageio.mimsave(output_gif, images, duration=duration, loop=0)
        print(f"GIF saved to {output_gif}")
        
        # Display the GIF
        display(Image(filename=output_gif))
    else:
        print("No images were processed to create the GIF.")
os.makedirs('gifs', exist_ok=True)
fire = active_fires['2018'][-1]
gif_name = '_'.join(fire.split('/')[-2:])+'.gif'
create_band_gif(fire, 22,labels[22], output_gif = f"gifs/{gif_name}")

# 2. Load the Model and Make Predictions

In [ ]:
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
# Initialize model architecture
model = ConvLSTM(1, 1, (5, 5), 1, True, True, False)
model.load_state_dict(torch.load("best.pth", map_location=device))
model.to(device)
model.eval()
TIME_STEPS = 5  # Number of leading observations
IMG_SIZE = (32, 32)
# INPUT_CHANNELS = 40
# OUTPUT_CHANNELS = 1
# EPOCHS = 50


## 2.1 Load Validation Dataset (can prob remove and put a single fire here)

In [ ]:
val_dataset = FireSpreadDataset(
    data_dir="/home/petark/PycharmProjects/WildfireSpreadTS/dataset_hdf5",  # Update with actual path
    included_fire_years=[2020],  # Validation year
    n_leading_observations=TIME_STEPS,
    crop_side_length=IMG_SIZE[0],
    load_from_hdf5=True,
    is_train=False,
    remove_duplicate_features=False,
    stats_years=[2018, 2019]
)

val_loader = DataLoader(val_dataset, batch_size=1, shuffle=False)

# 3. Classic el QUBO

In [ ]:
# Kotec enter here - we get a matrix which tells us where to shoot water

# 4. Quantum Optim.

In [ ]:
# Vlado enter here

# 5. Post Processing

In [ ]:
# ...